The following code cell ensures that the underlying `.py` file gets automatically reloaded after you save any changes.  

In [13]:
%load_ext autoreload
%autoreload 2

In [13]:
import pandas as pd    
from model import initialize_model, call_llm
import time
import json



In [8]:

def preprocess(file_path):
    data = pd.read_json(path_or_buf=file_path, lines=True)
    ids = set()
    rows_to_be_dropped = []
    for i in range(len(data)):
        row = data.iloc[i]
        if row.unique_id in ids:
            rows_to_be_dropped.append(i)
        else:
            ids.add(row.unique_id)
    data = data.drop(rows_to_be_dropped)
    return data

In [9]:
data = preprocess("./data/train.jsonl")

In [10]:
first_partition = data[:1365]
second_partition = data[1365:2730]
third_partition = data[2730:4095]
fourth_partition = data[4095:5460]
fifth_partition = data[5460:6825]
sixth_partition = data[6825:]

In [12]:
labels = []
reasonings = []
raw_output = []
ids = []
client, _ = initialize_model("")
last_time = time.time()

# Change len(data) to len(your_partition)
for i in range(1023, len(sixth_partition)):
    if i > 0 and i % 60 == 0:
        print(i)
        current_time = time.time()
        time.sleep(max(0, 63 - (current_time - last_time)))
        last_time = time.time()

    # Change data to your partition
    current_data = sixth_partition.iloc[i]
    ids.append(current_data.unique_id)
    classification, reasoning = call_llm(client, "", current_data.string, current_data.sectionName)
    raw_output.append(classification)
    labels.append(classification)
    reasonings.append(reasoning)
    

id='noC14fF-4Yz4kd-928e51320acdfd7c' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1743408069 model='meta-llama/Llama-3.3-70B-Instruct-Turbo-Free' choices=[ChatCompletionChoicesData(index=0, logprobs=None, seed=15564019382398300000, finish_reason=<FinishReason.StopSequence: 'stop'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='{"classification": "method", "reasoning": "The text describes a specific protocol and criteria for defining an ideal day for scheduling OPU, which suggests that it is describing the methodology used in the study, hence it is classified as a method section. Additionally, the section name \'Material and methods\' also supports this classification."}', tool_calls=[]))] prompt=[] usage=UsageData(prompt_tokens=218, completion_tokens=68, total_tokens=286)
id='noC15aW-3NKUce-928e51424a78fd7c' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1743408072 model='meta-llama/Llama-3.3-70B-Instruct-Turbo-Free' c

In [15]:
print(sixth_partition)

        source  citeEnd           sectionName  citeStart  \
6860  explicit    233.0               Results      208.0   
6861  explicit    176.0         7 EXPERIMENTS      172.0   
6862  explicit    210.0  3.2.2 Implementation      206.0   
6863  explicit      8.0             6 OUTLOOK        3.0   
6864  explicit    184.0            Discussion      164.0   
...        ...      ...                   ...        ...   
8238  explicit     50.0                             28.0   
8239  explicit    182.0            DISCUSSION      179.0   
8240  explicit    120.0            DISCUSSION      108.0   
8241  explicit    221.0                            185.0   
8242  explicit    102.0          Introduction       82.0   

                                                 string       label  \
6860  Work with non-neuronal cells has suggested tha...  background   
6861  We have added the affordances to the definitio...      method   
6862  To determine this canonical set of values, the...      metho

In [ ]:
filename = "full_output.txt"
c = []
r = []
i = [sixth_partition.iloc[x].unique_id for x in range(len(sixth_partition))]

with open(filename) as file:
    lines = [line.rstrip() for line in file]
    for line in lines:
        start = line.find("{")
        end = line.rfind("}") + 1
        json_str = json.dumps(eval(line[start:end]))
        
        answers = json.loads(json_str)
        
        classification = answers["classification"]
        reasoning = answers["reasoning"]
        print(classification)
        print(reasoning)
        c.append(classification)
        r.append(reasoning)

df = pd.DataFrame(zip(i, c, r), columns=["id", "model_classification", "reasoning"])
df.to_csv("your_partition.csv")

background
Although the section name is 'Results', the text provided is discussing previous research and its findings, which is a characteristic of a background section. It mentions a specific study (Vander Heiden et al., 2000, 2001) and explains its implications, setting the stage for the current research rather than presenting new results.
method
The text mentions 'EXPERIMENTS' which is a common term used in scientific papers to describe the methodology or approach used to test a hypothesis or validate results, indicating that this section is discussing the methods used in the research.
method
The text describes the implementation of a specific approach, mentioning the use of parameters from a mathematical model as a starting point, which suggests a step-by-step explanation of how something was done, typical of a methods section.
background
The text is classified as background because it appears to be a reference to a previous study or development, indicated by the phrase 'In [103]',

In [ ]:
df = pd.DataFrame(zip(ids, labels, reasonings), columns=["id", "model_classification", "reasoning"])
df.to_csv("fourth_partition_ethan.csv")